---
💡 AdaBoost: 핵심 원리 요약
---

AdaBoost(Adaptive Boosting)는 

- '약한 학습기(Weak Learner)'들을 `순차적으로 학습`시키면서 
- `오답에 집중하여 성능을 향상`시키는 앙상블 기법입니다.

---

가장 중요한 핵심은 다음과 같습니다.


1. 가중치 조정 (Adaptive Weighting):

     - 첫 번째 모델을 학습시키고 예측을 수행합니다.

     - `오답(잘못 예측한 데이터)에 더 높은 가중치를 부여`합니다.

     - 정답에 대해서는 가중치를 낮춥니다.

    - 이렇게 가중치가 조정된 데이터를 가지고 두 번째 모델을 학습시킵니다.

    - 이 과정을 반복하며, `매번 이전 모델이 틀린 문제에 더 집중`하는 새로운 모델을 만들어냅니다.

---

2. 투표 가중치 (Weighted Voting):

    - 이렇게 순차적으로 만들어진 모든 모델들이 최종 예측을 위해 투표합니다.

    - 이때, 성능이 좋았던 모델에게는 `더 많은 투표권(가중치)을 부여`하여 최종 결과에 더 큰 영향력을 행사하게 합니다.

    - 성능이 상대적으로 낮았던 모델은 투표권이 적습니다.

---

이 원리를 통해 AdaBoost는 `여러 개의 약한 모델을 결합`하여 강력하고 정확한 '하나의 모델'을 만들어냅니다.

---

 "`learning_rates`는 `낮게`, `n_estimators`는 `높게` 조합하여 모델의 성능을 극대화합니다"
 
 라는 부분이 바로 부스팅 모델을 튜닝할 때의 핵심 원리입니다.

---

* 이 조합은 모델이 오차를 수정하는 과정을 다음과 같이 만듭니다.

1. 낮은 learning_rate (신중한 학습): 한 번의 학습에서 오차를 크게 수정하지 않고 조금씩, 신중하게 배우게 합니다.

2. 높은 n_estimators (충분한 반복): 오차를 신중하게 수정하는 과정을 충분한 횟수만큼 반복하게 합니다.

---

왜 이 조합이 최적일까?

이 조합은 모델이 `과적합(Overfitting)을 피하면서`도 `복잡한 패턴을 정교하게 학습`할 수 있도록 돕습니다.


* 현업에서 부스팅 모델을 사용할 때는 

1. learning_rate는 낮게 설정하고, 
2. n_estimators는 높게 설정한 후 
3. 교차 검증(cross_val_score)을 통해 
4. 최적의 조합을 찾아내는 전략이 
5. 가장 보편적으로 사용됩니다.

In [10]:
# BOOT-09_AdaBoost_실습_해부.ipynb

# 1. 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

print("### AdaBoost 모델 학습 및 분석 실습 시작 ###")

# 2. 데이터셋 로드
# 타이타닉 학습 데이터를 불러옵니다.
df_train = pd.read_csv('../../titanic_project/titanic_train.csv')

# 3. 데이터 전처리
# 결측치를 처리하고, 모델 학습에 필요 없는 특성들을 제거합니다.
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())
df_train['Embarked'] = df_train['Embarked'].fillna(df_train['Embarked'].mode()[0])
df_train = df_train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, errors='ignore')

# 특성(X)과 레이블(y) 분리
X = df_train.drop('Survived', axis=1)
y = df_train['Survived']

# 학습 및 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 데이터 전처리 파이프라인 구축
# 수치형 데이터는 표준 스케일링을, 범주형 데이터는 원-핫 인코딩을 적용합니다.
categorical_features = ['Pclass', 'Sex', 'Embarked']
numerical_features = ['Age', 'SibSp', 'Parch', 'Fare']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

print("\n--- AdaBoost 모델 학습 및 성능 비교 시작 ---")

# 5. 첫 번째 AdaBoost 모델 (높은 learning_rate, 낮은 n_estimators)
# 이전 실습에서 가장 높은 정확도를 보인 조합입니다.
adaboost_model_high_lr_low_n = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1, random_state=42),
    n_estimators=50,
    learning_rate=1.0,
    random_state=42
)
adaboost_pipeline_high_lr_low_n = Pipeline(steps=[('preprocessor', preprocessor),
                                                  ('classifier', adaboost_model_high_lr_low_n)])

print("\n(1) learning_rate=1.0, n_estimators=50 모델 학습 중...")
adaboost_pipeline_high_lr_low_n.fit(X_train, y_train)
y_pred_high_lr_low_n = adaboost_pipeline_high_lr_low_n.predict(X_test)
accuracy_high_lr_low_n = accuracy_score(y_test, y_pred_high_lr_low_n)
print(f"✅ AdaBoost 모델 (높은 학습률, 단순한 트리) 정확도: {accuracy_high_lr_low_n:.4f}")


# 6. 두 번째 AdaBoost 모델 (낮은 learning_rate, 높은 n_estimators)
# 일반적으로 성능이 더 좋게 나올 것으로 기대되는 조합입니다.
adaboost_model_low_lr_high_n = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1, random_state=42),
    n_estimators=1000,
    learning_rate=0.1,
    random_state=42
)
adaboost_pipeline_low_lr_high_n = Pipeline(steps=[('preprocessor', preprocessor),
                                                  ('classifier', adaboost_model_low_lr_high_n)])

print("\n(2) learning_rate=0.1, n_estimators=1000 모델 학습 중...")
adaboost_pipeline_low_lr_high_n.fit(X_train, y_train)
y_pred_low_lr_high_n = adaboost_pipeline_low_lr_high_n.predict(X_test)
accuracy_low_lr_high_n = accuracy_score(y_test, y_pred_low_lr_high_n)
print(f"✅ AdaBoost 모델 (낮은 학습률, 단순한 트리) 정확도: {accuracy_low_lr_high_n:.4f}")

# 7. 세 번째 AdaBoost 모델 (max_depth를 높인 경우)
# 약한 학습기의 복잡도를 2로 높여서 성능 변화를 확인합니다.
adaboost_model_deeper = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=5, random_state=42), # max_depth를 1 -> 5로 변경
    n_estimators=1000,
    learning_rate=0.1,
    random_state=42
)
adaboost_pipeline_deeper = Pipeline(steps=[('preprocessor', preprocessor),
                                           ('classifier', adaboost_model_deeper)])

print("\n(3) max_depth=5, learning_rate=0.1, n_estimators=1000 모델 학습 중...")
adaboost_pipeline_deeper.fit(X_train, y_train)
y_pred_deeper = adaboost_pipeline_deeper.predict(X_test)
accuracy_deeper = accuracy_score(y_test, y_pred_deeper)
print(f"✅ AdaBoost 모델 (낮은 학습률, 복잡한 트리) 정확도: {accuracy_deeper:.4f}")


# 8. 모델의 중요 특징(feature importances) 확인
# 전처리 후 특성 이름들을 가져오는 과정입니다.
# StandardScaler와 OneHotEncoder를 거치면서 원래 특성 이름이 바뀌고 새로운 특성이 생기기 때문에
# 모든 특성 이름을 결합하는 과정이 필요합니다.
print("\n--- 특성 중요도(Feature Importance) 확인 ---")
# 전처리된 숫자형 특성 이름 가져오기
numerical_feature_names = numerical_features

# OneHotEncoder의 특성 이름 가져오기
onehot_encoded_names = adaboost_pipeline_deeper.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)

# 모든 특성 이름을 결합
all_feature_names = list(numerical_feature_names) + list(onehot_encoded_names)

feature_importances = adaboost_pipeline_deeper.named_steps['classifier'].feature_importances_
print("전처리된 특성 이름:", all_feature_names)
print("특성 중요도:", feature_importances)


### AdaBoost 모델 학습 및 분석 실습 시작 ###

--- AdaBoost 모델 학습 및 성능 비교 시작 ---

(1) learning_rate=1.0, n_estimators=50 모델 학습 중...
✅ AdaBoost 모델 (높은 학습률, 단순한 트리) 정확도: 0.7989

(2) learning_rate=0.1, n_estimators=1000 모델 학습 중...
✅ AdaBoost 모델 (낮은 학습률, 단순한 트리) 정확도: 0.7989

(3) max_depth=5, learning_rate=0.1, n_estimators=1000 모델 학습 중...
✅ AdaBoost 모델 (낮은 학습률, 복잡한 트리) 정확도: 0.8156

--- 특성 중요도(Feature Importance) 확인 ---
전처리된 특성 이름: ['Age', 'SibSp', 'Parch', 'Fare', 'Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S']
특성 중요도: [0.31661433 0.05797343 0.06979088 0.33199116 0.01551255 0.01444374
 0.06335676 0.04183031 0.05930674 0.01853239 0.00051885 0.01012886]
